<a href="https://colab.research.google.com/github/rescobarcorrea/ArcGIS_server_managment/blob/main/changeRecyclingTimesAdminAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Various iterable functions for the rest API of arcGIS
<Rogger.Correa>

In [ ]:
pip install retry  

In [ ]:
# Imports


#from collections import OrderedDict
import json
import urllib
import urllib.request
import numpy as np
from google.colab import files
from datetime import datetime
from datetime import timedelta
from retry.api import retry_call
import logging
import ssl

In [ ]:
#Inputs
server='https://awsgis.fcnsw.com.au'
username="user"
password="password"# Chage this your own password
Ip=34.91.146.221
#optionals
tries = 20 #number of times that request a url
secondsoftry=5#seconds watiing between tries
expiration=60

#derivated inputs
#admintokenurl
admin_token_url=server+"/arcgis/admin/generateToken"


#Logger details
logger = logging.getLogger('server_admin_api_log.log')
logging.basicConfig(level=logging.INFO)

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
#python 3.7


In [ ]:
#Functions *************************************
# functions functions functions, get your functions here!
class TokenError(Exception):
  'Invalid Token or busy server, retry again'
  pass

def catchTokenError (errormessage):
  if errormessage=='Invalid Token' or errormessage=='Invalid token, ClientID does not match.':
          raise TokenError('Invalid Token or busy server, retry again')
  else:
          #raise
          result=errormessage
          logger.error('not a token error')
  return result

def gentoken(url, username, password, expiration=60):
    #Get your token here
    #this funtion now wokring, note that you need to be admin of the server
    query_dict = {'username':   username,
                  'password':   password,
                  'expiration': str(expiration),
                  'client':     'requestip'}
    query_string = urllib.parse.urlencode(query_dict)
    query_string=query_string.encode('ascii')

    token=json.loads(urllib.request.urlopen(url + "?f=json", query_string,context=ctx).read())['token']
    
    if  not token:
        raise
        logging.error('token not generated')
        quit()
    else:
        return token

def GetServiceConections(service_url):
  #this function gets the GDb coenctions of each service feature layer
    metadata_url = '{0}/iteminfo/manifest/manifest.xml'.format(service_url)
    xml_data = requests.get(metadata_url,params={'token':token}).content
    data = dict(xmltodict.parse(xml_input=xml_data,encoding='UTF-8'))
    service_items = []
    datasets = data['SVCManifest']['Databases']['SVCDatabase']['Datasets']['SVCDataset']
    if isinstance(datasets,list):
        for ds in datasets:
          servicerow=[service_url,ds['Name'], ds['OnPremisePath'].split('\\')[-1]]
          service_items.append(servicerow)
    elif isinstance(datasets,OrderedDict):
        servicerow=[service_url,datasets['Name'], datasets['OnPremisePath'].split('\\')[-1]]
        service_items.append(servicerow)
    return service_items

def getServiceDictionary(service_admin_url,token):
  #function that update the featureservercapabilities

  #the service url has to be the one located in the admin folder
  #eg. https://<server>/arcgis/admin/services/<folder>/<service>.MapServer
  # Feature server capabilities has to be like this:
  #newFScapabilities="Create,Delete,Extract,Query,Sync,Update,Uploads,Editing"

  # Here I read the old service dict
  servicedicturl=service_admin_url+ "?f=json&token=" + token
  response=urllib.request.urlopen(servicedicturl,context=ctx)
  jsonResponse=json.load(response)
  try:
        errormessage=jsonResponse['error']
        if errormessage['message']=='Invalid Token':
          raise TokenError('Invalid Token or busy server, retry again')
        else:
          result='error reading service dictionary'
          logger.error(result)
  except KeyError:
        #print(TokenError.error)
        result=jsonResponse
  return result


def changeFScapabilities(baseUrl,folder,servicename,servicetype,newFScapabilities,token):
  #function that update the featureservercapabilities

  #the service url has to be the one located in the admin folder
  #eg. https://<server>/arcgis/admin/services/<folder>/<service>.MapServer
  # Feature server capabilities has to be like this:
  #newFScapabilities="Create,Delete,Extract,Query,Sync,Update,Uploads,Editing"

  # Here I read the old service dict
  servicedicturl=baseUrl+'/'+folder+'/'+servicename+servicetype+ "?f=json&token=" + token
  response=urllib.request.urlopen(servicedicturl,context=ctx)
  old_service_dict = json.load(response)

  #Here I change the parameters I want in a new dict
  new_service_dict=old_service_dict# make a copy of the old dic
  #here go the parameters to  update, so far only 2 just to test

  #changes description 
  #newdescription='I\'m chanding descriptions from code in puthon'
  #new_service_dict['description']=newdescription

  #changes feature service capabilities
  new_service_dict['extensions'][7]['capabilities']=newFScapabilities

  params = urllib.parse.urlencode({'service'  : new_service_dict,'token' : token, 'f' : 'json'})
  params=params.encode('UTF8')
  editurl=baseUrl+'/'+folder+'/'+servicename+servicetype+'/edit'#editurl
  response = urllib.request.urlopen(editurl, params).read()
  jsonResponse=json.loads(response)
  try:
        errormessage=jsonResponse['error']
        if errormessage['message']=='Invalid Token':
          raise TokenError('Invalid Token or busy server, retry again')
        else:
          result='error changing capabilities'
          logger.error(errormessage['message'])
  except KeyError:
        #print(TokenError.error)
        result=jsonResponse
  return result
  
def changeFSrecyclingtime(service_admin_url,newFSrecyclingtime,token):
  #function that update the featureservercapabilities

  #the service url has to be the one located in the admin folder
  #eg. https://<server>/arcgis/admin/services/<folder>/<service>.MapServer
  # Feature server capabilities has to be like this:
  #newFScapabilities="Create,Delete,Extract,Query,Sync,Update,Uploads,Editing"
  logger.warning('...Editing service...')
  # Here I read the old service dict and make a copy of it to change
  New_Service_Dictionary=getServiceDictionary(service_admin_url,token)
  
  #changes feature service recyclingtime
  New_Service_Dictionary['recycleStartTime']=newFSrecyclingtime
  #uploads
  params = urllib.parse.urlencode({'service'  : New_Service_Dictionary,'token' : token, 'f' : 'json'})
  params=params.encode('UTF8')
  editurl=service_admin_url+'/edit'#editurl
  response = urllib.request.urlopen(editurl, params,context=ctx).read()
  jsonResponse=json.loads(response)
  if 'error' in jsonResponse:
      logger.warning('error in catalog rest')
      errormessage=catalog['error']['message']  
      try:
        catchTokenError(errormessage)
        
      except KeyError:
          #print(TokenError.error)
          result=catalog
          return result
      
  elif 'status' in jsonResponse:
    if 'error' in jsonResponse['status']:
          
      logger.warning('error in catalog admin')
      errormessage=jsonResponse['messages'][0]
      logger.error(errormessage)  
      try:
        catchTokenError(errormessage)
      
      except KeyError:
        #print(TokenError.error)
        result=jsonResponse
        return result
    elif 'success' in jsonResponse['status']:
      result='Successfully changed!'
      return result
      #logger.warning(result)
  else:
      result=jsonResponse
      return result


def getStatistics(service_url,token):
  #a function to read how much a service has been used
  #the service url has to be the one located in the admin folder
  #eg. https://<server>/arcgis/admin/services/<folder>/<service>.MapServer
  servicestatisticsurl=service_url+ "/statistics?f=json&token=" + token
  response=urllib.request.urlopen(servicestatisticsurl)
  statistics=json.load(response)
  #starttime=statistics['summary']['startTime']
  transactions=statistics['summary']['transactions']
  totalBusyTime=statistics['summary']['totalBusyTime']
  return transactions,totalBusyTime


def getServicesList(baserestUrl,token,folderName=None):
  logger.warning('...requesting services manifesto')
  if not folderName:
    catalog=json.load(urllib.request.urlopen(baserestUrl + "/" + "?f=json&token=" + token,context=ctx))
  else:
    catalog=json.load(urllib.request.urlopen(baserestUrl + "/" + folderName + "?f=json&token=" + token,context=ctx))
  if 'error' in catalog:
    logger.warning('error in catalog rest')
    errormessage=catalog['error']['message']  
    try:
       catchTokenError(errormessage)
       
    except KeyError:
        #print(TokenError.error)
        result=catalog
        return result
    
  elif 'status' in catalog:
    logger.warning('error in catalog admin')
    errormessage=catalog['messages'][0]
    #logger.error(errormessage)  
    try:
       catchTokenError(errormessage)
     
    except KeyError:
       #print(TokenError.error)
       result=catalog
       return result
  else:
    result=catalog
    return result
function='changeFSrecyclingtime'
fargs='startTime,timeincrement,token,tries=20,secondsoftry=5'

def changeAllservicesrecyclingtime(baseServerUrl,startTime,timeincrement,token,tries=20,secondsoftry=5):
  #This function needs all the previous functiosn to operate. 
  #It loops throw a server services and test it's syncing
  #convertstime
  timeFormat='%H:%M'
  starttimestamp=datetime.strptime(startTime, timeFormat)
  #Read manifesto
  
  catalog=retry_call(getServicesList,fargs=[baseServerUrl,token],tries=tries,delay=secondsoftry)
  services=catalog['services']
  service_urls=[]
  counter=0
  folderName=''
  for service in services:
      serviceurl=baseServerUrl+ '/' + service['serviceName'] + '.' + service['type']
      if service['type']=='MapServer':
        
        logger.info('   *****   '+str(counter)+'----'+service['serviceName']+'    *****   ')
        ## Here, put a function

        #time stuff
        finaltime=starttimestamp+timedelta(minutes=timeincrement)
        starttimestamp=finaltime
        newFSrecyclingtime=finaltime.strftime(timeFormat)
        logger.info('changing time to: '+newFSrecyclingtime)

        try:  
          changed=retry_call(changeFSrecyclingtime,fargs=[serviceurl,newFSrecyclingtime,token],tries=tries,delay=secondsoftry)
          logger.info('-----' +service['serviceName']+'-----' +changed)
        except:
          logger.info('-----' +service['serviceName']+'-----' +'Not changed after '+str(tries)+' attepmts')
          continue
        
     
  if 'folders' in catalog:
    folders=catalog['folders']
    folders
    for folderName in folders:
      logger.info(folderName)
      catalog=retry_call(getServicesList,fargs=[baseServerUrl,token],fkwargs={'folderName': folderName},tries=tries,delay=secondsoftry)
      services=catalog['services']
      for service in services:
                  counter+=1;
                  serviceurl=baseServerUrl+ '/' + folderName +'/'+ service['serviceName'] + '.' + service['type']
                  if service['type']=='MapServer':
                          ## Here put a function
                          
                          logger.info('   *****   '+str(counter)+'----'+service['serviceName']+'    *****   ')
                          #time stuff
                          finaltime=starttimestamp+timedelta(minutes=timeincrement)
                          starttimestamp=finaltime
                          newFSrecyclingtime=finaltime.strftime(timeFormat)
                          logger.info('changing time to: '+newFSrecyclingtime)
           
                          try:  
                            changed=retry_call(changeFSrecyclingtime,fargs=[serviceurl,newFSrecyclingtime,token],tries=tries,delay=secondsoftry)
                            logger.info('-----' +service['serviceName']+'-----' +changed)
                          except (TypeError):
                            logger.info('-----' +service['serviceName']+'-----' +'Not changed after '+str(tries)+' attepmts')
                            continue
#END OF FUNCTIONS

In [ ]:
#Function to get services statistics

def getAllservicesStatistics(baseServerUrl,token,tries=20,secondsoftry=5):
  #This function needs all the previous functiosn to operate. 
  #It loops throw a server services and test it's syncing
  #convertstime
  timeFormat='%H:%M'
  
  #Read manifesto
  
  catalog=retry_call(getServicesList,fargs=[baseServerUrl,token],tries=tries,delay=secondsoftry)
  services=catalog['services']
  service_urls=[]
  counter=0
  folderName=''
  for service in services:
      serviceurl=baseServerUrl+ '/' + service['serviceName'] + '.' + service['type']
      if service['type']=='MapServer':
        
        logger.info('   *****   '+str(counter)+'----'+service['serviceName']+'    *****   ')
        ## Here, put a function
        try:  
          changed=retry_call(getStatistics,fargs=[serviceurl,token],tries=tries,delay=secondsoftry)
          logger.info('-----' +service['serviceName']+'-----' +changed)
        except:
          logger.info('-----' +service['serviceName']+'-----' +'Not changed after '+str(tries)+' attepmts')
          continue
        
     
  if 'folders' in catalog:
    folders=catalog['folders']
    folders
    for folderName in folders:
      logger.info(folderName)
      catalog=retry_call(getServicesList,fargs=[baseServerUrl,token],fkwargs={'folderName': folderName},tries=tries,delay=secondsoftry)
      services=catalog['services']
      for service in services:
                  counter+=1;
                  serviceurl=baseServerUrl+ '/' + folderName +'/'+ service['serviceName'] + '.' + service['type']
                  if service['type']=='MapServer':
                          ## Here put a function
                          
                          logger.info('   *****   '+str(counter)+'----'+service['serviceName']+'    *****   ')
                          #time stuff
                   
                          try:  
                            changed=retry_call(getStatistics,fargs=[serviceurl,token],tries=tries,delay=secondsoftry)
                            logger.info('-----' +service['serviceName']+'-----' +changed)
                          except (TypeError):
                            logger.info('-----' +service['serviceName']+'-----' +'Not changed after '+str(tries)+' attepmts')
                            continue
#END OF FUNCTIONS

In [ ]:
#Iterate function over all services
function='changeFSrecyclingtime'
fargs='startTime,timeincrement,token,tries=20,secondsoftry=5'



In [ ]:
#main to use For a single service
folder='/FCNSWcorp'
servicename='/AOIsOnDemand'#Put here an specific service name

#derivated inputs
baseadminUrl=server+"/arcgis/admin/services"
service_admin_url=server+"/arcgis/admin/services"+folder+servicename+'.MapServer'

newFSrecyclingtime='15:00'

#MAIN:  for a single service
token=gentoken(admin_token_url,username,password)
changed=retry_call(changeFSrecyclingtime,fargs=[service_admin_url,newFSrecyclingtime,token],tries=tries,delay=secondsoftry)
print(changed)

Successfully changed!


In [ ]:
eval('startTime,timeincrement')

('10:00', 5)

In [ ]:
#main to use For a whole server
#get IP !curl ipecho.net/plain
startTime='10:00' #string
timeincrement=5 #minutes in integer
baseadminUrl=server+"/arcgis/admin/services"
token=gentoken(admin_token_url,username,password)
fucntion=('')
changeAllservicesrecyclingtime(baseadminUrl,startTime,timeincrement,token,tries=20,secondsoftry=5)

In [ ]:
ip=!curl ipecho.net/plain
print(ip)

['34.91.146.221']


In [ ]:
baseadminUrl=server+"/arcgis/admin/services"
token=gentoken(admin_token_url,username,password)

getAllservicesStatistics(baseadminUrl,token,tries=20,secondsoftry=5)

KeyboardInterrupt: ignored

In [ ]:
toke="xQ3fnhP7eDl6MpTDD1ZT8iyPfB1zW8kuYJryJB8Lnq79b4JAmPRSYdfsgiqW69rJ"
getAllservicesStatistics(baseadminUrl,toke,tries=20,secondsoftry=5)

URLError: ignored

In [ ]:
#Download Log

files.download('server_admin_api_log.log')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>